# SISO System Identification for Ground Motion Events

In [1]:
import numpy as np
from utilities.printing import *
import ssid
from ssid import modes
from control.matlab import ss, lsim
from collections import defaultdict
import sdof
from quakeio.elcentro import accel as elcentro

## Upload a Dataset and Identify Spectral Quantities

In [2]:
input = np.loadtxt("./uploads/opensees_sdof/Accgrd_set1.csv", delimiter=",")
output = np.loadtxt("./uploads/opensees_sdof/Accrsp_set1.csv", delimiter=",")
# input = np.loadtxt("./uploads/opensees_sdof/Accgrd_set2.csv", delimiter=",")
# output = np.loadtxt("./uploads/opensees_sdof/Accrsp_set2.csv", delimiter=",")
t = input[0,:]
dt = t[1] - t[0]

### Method Inputs

See [`01_SISO_Intro`](01_SISO_Intro) for parameter definitions

In [ ]:
# Set parameters
conf = {
    "m":  300,
    "no": 140,
    "nc": 140,
    "r":  2,
    "a":  0,
    "b":  0,
    "l":  10,
    "g":  3,
    "ns": 140
}

### Perform System Identification

In [5]:
spred = np.empty((input.shape[0]-1, 2))

for i,motion in enumerate(input[1:,:]):
    A,B,C,D = ssid.system(method="srim", input=motion, output=output[i+1,:], **conf)
    ss_modes = modes.modes((A,B,C,D),dt)
    spred[i,:] = [[1/v["freq"], v["damp"]] for v in ss_modes.values()][0]  # save predicted period and damping


print("   period(s)  damping\n", spred)
np.savetxt("output/set1_srim.csv", spred)

   period(s)  damping
 [[0.41080027 0.02340861]
 [0.41080028 0.02340847]
 [0.41080027 0.02340856]
 [0.41080019 0.02340856]
 [0.41080029 0.02340851]
 [0.41080029 0.02340858]
 [0.41080021 0.0234084 ]
 [0.41080027 0.02340857]
 [0.41080023 0.02340843]
 [0.41080023 0.02340838]
 [0.41080029 0.02340854]]
